In [1]:
# 1.0 Importação das bibliotecas
import duckdb

# Criando conexão com DuckDB
conn = duckdb.connect()

print(" DuckDB configurado com sucesso!")
print(f"Versão do DuckDB: {duckdb.__version__}")

 DuckDB configurado com sucesso!
Versão do DuckDB: 1.3.2


In [2]:
#2.0 Validação do Dataset e Inspeção Inicial
# Definindo o caminho do dataset
dataset_path = '/kaggle/input/imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv'

# Query para validar acesso ao arquivo e inspecionar schema
query_schema = f"""
SELECT * 
FROM read_csv_auto('{dataset_path}')
LIMIT 5;
"""

# Executando a query
resultado = conn.execute(query_schema).fetchdf()
print("Primeiras 5 linhas do dataset:")
print(resultado)
# Query para obter informações sobre as colunas do dataset
query_info = f"""
DESCRIBE SELECT * FROM read_csv_auto('{dataset_path}');
"""

info = conn.execute(query_info).fetchdf()
print("Estrutura do Dataset:")
print(info)
# Query para contar total de registros
query_count = f"""
SELECT COUNT(*) as total_registros
FROM read_csv_auto('{dataset_path}');
"""

total = conn.execute(query_count).fetchdf()
print(f"Total de registros no dataset: {total['total_registros'].values[0]}")

Primeiras 5 linhas do dataset:
                                         Poster_Link  \
0  https://m.media-amazon.com/images/M/MV5BMDFkYT...   
1  https://m.media-amazon.com/images/M/MV5BM2MyNj...   
2  https://m.media-amazon.com/images/M/MV5BMTMxNT...   
3  https://m.media-amazon.com/images/M/MV5BMWMwMG...   
4  https://m.media-amazon.com/images/M/MV5BMWU4N2...   

               Series_Title Released_Year Certificate  Runtime  \
0  The Shawshank Redemption          1994           A  142 min   
1             The Godfather          1972           A  175 min   
2           The Dark Knight          2008          UA  152 min   
3    The Godfather: Part II          1974           A  202 min   
4              12 Angry Men          1957           U   96 min   

                  Genre  IMDB_Rating  \
0                 Drama          9.3   
1          Crime, Drama          9.2   
2  Action, Crime, Drama          9.0   
3          Crime, Drama          9.0   
4          Crime, Drama          9.

In [3]:
#3.0 Consultas SQL Fundamentais
#3.1 Seleção Básica com Filtros e Ordenação
# Objetivo: Listar os 10 filmes mais recentes com nota IMDb acima de 8.5
# Conceitos: SELECT, WHERE, ORDER BY, LIMIT

query_top_recent = f"""
SELECT 
    Series_Title AS titulo,
    Released_Year AS ano_lancamento,
    IMDB_Rating AS nota_imdb,
    Genre AS genero,
    Director AS diretor
FROM read_csv_auto('{dataset_path}')
WHERE IMDB_Rating > 8.5
    AND Released_Year != 'PG'  -- Filtrando valores inválidos na coluna de ano
ORDER BY Released_Year DESC, IMDB_Rating DESC
LIMIT 10;
"""

resultado = conn.execute(query_top_recent).fetchdf()
print("Top 10 Filmes Mais Recentes com Nota > 8.5:")
print(resultado.to_string(index=False))

Top 10 Filmes Mais Recentes com Nota > 8.5:
                                           titulo ano_lancamento  nota_imdb                    genero            diretor
                                  Soorarai Pottru           2020        8.6                     Drama      Sudha Kongara
                                         Hamilton           2020        8.6 Biography, Drama, History        Thomas Kail
                                     Gisaengchung           2019        8.6   Comedy, Drama, Thriller       Bong Joon Ho
                                     Interstellar           2014        8.6  Adventure, Drama, Sci-Fi  Christopher Nolan
                                        Inception           2010        8.8 Action, Adventure, Sci-Fi  Christopher Nolan
                                  The Dark Knight           2008        9.0      Action, Crime, Drama  Christopher Nolan
    The Lord of the Rings: The Return of the King           2003        8.9  Action, Adventure, Drama      Pe

In [4]:
#3.2 Filtragem por Gênero e Diretor
# Objetivo: Encontrar filmes de ação dirigidos por Christopher Nolan
# Conceitos: WHERE com múltiplas condições, LIKE para pattern matching

query_nolan_action = f"""
SELECT 
    Series_Title AS titulo,
    Released_Year AS ano,
    IMDB_Rating AS nota,
    Genre AS genero,
    Director AS diretor
FROM read_csv_auto('{dataset_path}')
WHERE Director = 'Christopher Nolan'
    AND Genre LIKE '%Action%'
ORDER BY IMDB_Rating DESC;
"""

resultado = conn.execute(query_nolan_action).fetchdf()
print("Filmes de Ação de Christopher Nolan:")
print(resultado.to_string(index=False))

Filmes de Ação de Christopher Nolan:
               titulo  ano  nota                    genero           diretor
      The Dark Knight 2008   9.0      Action, Crime, Drama Christopher Nolan
            Inception 2010   8.8 Action, Adventure, Sci-Fi Christopher Nolan
The Dark Knight Rises 2012   8.4         Action, Adventure Christopher Nolan
        Batman Begins 2005   8.2         Action, Adventure Christopher Nolan
              Dunkirk 2017   7.8    Action, Drama, History Christopher Nolan


In [5]:
#4.0 Agregações e Métricas
#4.1 Estatísticas por Gênero
# Objetivo: Calcular estatísticas agregadas por gênero principal
# Conceitos: GROUP BY, COUNT, AVG, MIN, MAX, ROUND

query_stats_genre = f"""
SELECT 
    SPLIT_PART(Genre, ',', 1) AS genero_principal,
    COUNT(*) AS total_filmes,
    ROUND(AVG(IMDB_Rating), 2) AS nota_media,
    MIN(IMDB_Rating) AS nota_minima,
    MAX(IMDB_Rating) AS nota_maxima,
    ROUND(AVG(Meta_score), 2) AS metascore_medio
FROM read_csv_auto('{dataset_path}')
WHERE Genre IS NOT NULL
GROUP BY genero_principal
HAVING COUNT(*) >= 10  -- Apenas gêneros com pelo menos 10 filmes
ORDER BY nota_media DESC;
"""

resultado = conn.execute(query_stats_genre).fetchdf()
print("Estatísticas por Gênero (mínimo 10 filmes):")
print(resultado.to_string(index=False))

Estatísticas por Gênero (mínimo 10 filmes):
genero_principal  total_filmes  nota_media  nota_minima  nota_maxima  metascore_medio
           Crime           107        8.02          7.6          9.2            77.08
         Mystery            12        7.97          7.6          8.4            79.13
           Drama           289        7.96          7.6          9.3            79.70
          Action           172        7.95          7.6          9.0            73.42
       Adventure            72        7.94          7.6          8.6            78.44
       Biography            88        7.94          7.6          8.9            76.24
       Animation            82        7.93          7.6          8.6            81.09
          Horror            11        7.91          7.6          8.5            80.00
          Comedy           155        7.90          7.6          8.6            78.72


In [6]:
# 4.2 Análise por Certificação (Classificação Etária)
# Objetivo: Analisar distribuição e médias por certificação
# Conceitos: Agregação com GROUP BY, filtragem e casting de tipos

query_cert_analysis = f"""
SELECT
    Certificate AS classificacao,
    COUNT(*) AS quantidade,
    ROUND(AVG(IMDB_Rating), 2) AS nota_media_imdb,
    ROUND(
        AVG(
            CAST(REPLACE(Gross, ',', '') AS DOUBLE)
        ),
        2
    ) AS receita_media,
    ROUND(
        AVG(
            CAST(REPLACE(Runtime, ' min', '') AS DOUBLE)
        ),
        2
    ) AS duracao_media_min
FROM read_csv_auto('{dataset_path}')
WHERE Certificate IS NOT NULL
  AND Certificate != ''
  AND Gross IS NOT NULL
  AND Runtime IS NOT NULL
GROUP BY Certificate
ORDER BY quantidade DESC;
"""

resultado = conn.execute(query_cert_analysis).fetchdf()

print("Análise por Classificação Etária:")
print(resultado.to_string(index=False))

Análise por Classificação Etária:
classificacao  quantidade  nota_media_imdb  receita_media  duracao_media_min
            U         200             7.97    87246580.65             124.02
            A         180             8.00    63887132.82             128.81
           UA         163             7.95   131145954.52             129.71
            R         135             7.87    25793599.39             122.22
        PG-13          39             7.78    36947857.08             115.54
           PG          24             7.90    17888273.29             111.63
       Passed          23             8.06     5435553.74             107.09
            G          10             8.03    49596325.70             114.20
     Approved           6             8.02     6949014.00             124.67
           GP           2             7.85      708958.00             124.50
      Unrated           1             8.10      300000.00             183.00
          U/A           1             7.60

In [7]:
#4.3 Diretores Mais Produtivos
# Objetivo: Identificar diretores com mais filmes no top 1000
# Conceitos: Agregação, ordenação por contagem

query_top_directors = f"""
SELECT 
    Director AS diretor,
    COUNT(*) AS total_filmes,
    ROUND(AVG(IMDB_Rating), 2) AS nota_media,
    MIN(Released_Year) AS primeiro_filme,
    MAX(Released_Year) AS ultimo_filme
FROM read_csv_auto('{dataset_path}')
WHERE Director IS NOT NULL
GROUP BY Director
HAVING COUNT(*) >= 5  -- Diretores com pelo menos 5 filmes
ORDER BY total_filmes DESC, nota_media DESC
LIMIT 15;
"""

resultado = conn.execute(query_top_directors).fetchdf()
print("Diretores Mais Produtivos (mínimo 5 filmes):")
print(resultado.to_string(index=False))

Diretores Mais Produtivos (mínimo 5 filmes):
          diretor  total_filmes  nota_media primeiro_filme ultimo_filme
 Alfred Hitchcock            14        8.01           1935         1963
 Steven Spielberg            13        8.03           1975         2015
   Hayao Miyazaki            11        8.02           1979         2013
   Akira Kurosawa            10        8.22           1950         1985
  Martin Scorsese            10        8.17           1976         2019
  Stanley Kubrick             9        8.23           1956         1987
     Billy Wilder             9        8.14           1944         1960
      Woody Allen             9        7.79           1975         2011
Christopher Nolan             8        8.46           2000         2017
Quentin Tarantino             8        8.18           1992         2019
    David Fincher             8        8.04           1995         2014
   Clint Eastwood             8        7.91           1976         2008
   Ingmar Bergman  

In [8]:
# 5. Subqueries e CTEs
# 5.1 Filmes Acima da Média por Gênero (usando Subquery)
# Objetivo: Encontrar filmes com nota acima da média de seu gênero principal
# Conceitos: Subquery correlacionada, comparação com agregação

query_above_avg = f"""
SELECT 
    Series_Title AS titulo,
    SPLIT_PART(Genre, ',', 1) AS genero,
    IMDB_Rating AS nota,
    Released_Year AS ano
FROM read_csv_auto('{dataset_path}') AS f1
WHERE IMDB_Rating > (
    SELECT AVG(IMDB_Rating)
    FROM read_csv_auto('{dataset_path}') AS f2
    WHERE SPLIT_PART(f2.Genre, ',', 1) = SPLIT_PART(f1.Genre, ',', 1)
)
  AND Released_Year NOT IN ('PG', 'TV-MA', 'Approved')
ORDER BY genero, IMDB_Rating DESC
LIMIT 20;
"""

resultado = conn.execute(query_above_avg).fetchdf()

print("Filmes Acima da Média de seu Gênero:")
print(resultado.to_string(index=False))

Filmes Acima da Média de seu Gênero:
                                           titulo genero  nota  ano
                                  The Dark Knight Action   9.0 2008
    The Lord of the Rings: The Return of the King Action   8.9 2003
                                        Inception Action   8.8 2010
The Lord of the Rings: The Fellowship of the Ring Action   8.8 2001
                                       The Matrix Action   8.7 1999
   Star Wars: Episode V - The Empire Strikes Back Action   8.7 1980
            The Lord of the Rings: The Two Towers Action   8.7 2002
                                          Seppuku Action   8.6 1962
                             Shichinin no samurai Action   8.6 1954
                                        Star Wars Action   8.6 1977
                                             Léon Action   8.5 1994
                       Terminator 2: Judgment Day Action   8.5 1991
                                        Gladiator Action   8.5 2000
           

In [9]:
# 5.2 Análise por Década com CTE
# Objetivo: Agrupar filmes por década e calcular métricas
# Conceitos: CTE (WITH), CASE WHEN, agregações complexas

query_decade_analysis = f"""
WITH filmes_decada AS (
    SELECT 
        Series_Title,
        CAST(Released_Year AS INTEGER) AS ano_numerico,
        CASE 
            WHEN CAST(Released_Year AS INTEGER) >= 1920 AND CAST(Released_Year AS INTEGER) < 1930 THEN '1920s'
            WHEN CAST(Released_Year AS INTEGER) >= 1930 AND CAST(Released_Year AS INTEGER) < 1940 THEN '1930s'
            WHEN CAST(Released_Year AS INTEGER) >= 1940 AND CAST(Released_Year AS INTEGER) < 1950 THEN '1940s'
            WHEN CAST(Released_Year AS INTEGER) >= 1950 AND CAST(Released_Year AS INTEGER) < 1960 THEN '1950s'
            WHEN CAST(Released_Year AS INTEGER) >= 1960 AND CAST(Released_Year AS INTEGER) < 1970 THEN '1960s'
            WHEN CAST(Released_Year AS INTEGER) >= 1970 AND CAST(Released_Year AS INTEGER) < 1980 THEN '1970s'
            WHEN CAST(Released_Year AS INTEGER) >= 1980 AND CAST(Released_Year AS INTEGER) < 1990 THEN '1980s'
            WHEN CAST(Released_Year AS INTEGER) >= 1990 AND CAST(Released_Year AS INTEGER) < 2000 THEN '1990s'
            WHEN CAST(Released_Year AS INTEGER) >= 2000 AND CAST(Released_Year AS INTEGER) < 2010 THEN '2000s'
            WHEN CAST(Released_Year AS INTEGER) >= 2010 AND CAST(Released_Year AS INTEGER) < 2020 THEN '2010s'
            ELSE '2020s'
        END AS decada,
        IMDB_Rating,
        Meta_score,
        No_of_Votes
    FROM read_csv_auto('{dataset_path}')
    WHERE Released_Year ~ '^[0-9]{{4}}$'  -- Apenas anos válidos
)
SELECT 
    decada,
    COUNT(*) AS total_filmes,
    ROUND(AVG(IMDB_Rating), 2) AS nota_media,
    ROUND(AVG(Meta_score), 2) AS metascore_medio,
    ROUND(AVG(No_of_Votes), 0) AS votos_medios
FROM filmes_decada
GROUP BY decada
ORDER BY decada DESC;
"""

resultado = conn.execute(query_decade_analysis).fetchdf()
print("Análise por Década:")
print(resultado.to_string(index=False))

Análise por Década:
decada  total_filmes  nota_media  metascore_medio  votos_medios
 2020s             6        8.13            82.75       84127.0
 2010s           242        7.92            77.31      315622.0
 2000s           237        7.90            73.74      346422.0
 1990s           150        7.96            74.14      355100.0
 1980s            89        7.95            77.07      260641.0
 1970s            76        7.97            81.17      207811.0
 1960s            73        7.97            83.05      122554.0
 1950s            56        8.06            90.38      118288.0
 1940s            35        8.03            92.45      103858.0
 1930s            24        7.97            90.80       89730.0
 1920s            11        8.13            95.00       74684.0


In [10]:
# 5.3 CTE Múltiplas: Comparação de Períodos
# Objetivo: Comparar filmes clássicos (antes 1980) vs modernos (depois 2000)
# Conceitos: Múltiplas CTEs, UNION, agregações comparativas

query_period_comparison = f"""
WITH classicos AS (
    SELECT 
        'Clássicos (até 1979)' AS periodo,
        COUNT(*) AS quantidade,
        ROUND(AVG(IMDB_Rating), 2) AS nota_media,
        ROUND(AVG(Meta_score), 2) AS metascore_medio
    FROM read_csv_auto('{dataset_path}')
    WHERE Released_Year ~ '^[0-9]{{4}}$'
      AND CAST(Released_Year AS INTEGER) < 1980
),
modernos AS (
    SELECT 
        'Modernos (2000+)' AS periodo,
        COUNT(*) AS quantidade,
        ROUND(AVG(IMDB_Rating), 2) AS nota_media,
        ROUND(AVG(Meta_score), 2) AS metascore_medio
    FROM read_csv_auto('{dataset_path}')
    WHERE Released_Year ~ '^[0-9]{{4}}$'
      AND CAST(Released_Year AS INTEGER) >= 2000
)
SELECT * FROM classicos
UNION ALL
SELECT * FROM modernos;
"""

resultado = conn.execute(query_period_comparison).fetchdf()

print("Comparação: Filmes Clássicos vs Modernos:")
print(resultado.to_string(index=False))

Comparação: Filmes Clássicos vs Modernos:
             periodo  quantidade  nota_media  metascore_medio
Clássicos (até 1979)         275        8.00            85.68
    Modernos (2000+)         485        7.91            75.54


In [11]:
#6.0 Window Functions (Funções de Janela)
#6.1 Ranking Global de Filmes
# Objetivo: Criar ranking de filmes por nota IMDb
# Conceitos: ROW_NUMBER, RANK, DENSE_RANK - diferenças entre eles

query_global_ranking = f"""
SELECT 
    Series_Title AS titulo,
    Released_Year AS ano,
    IMDB_Rating AS nota,
    No_of_Votes AS votos,
    ROW_NUMBER() OVER (ORDER BY IMDB_Rating DESC, No_of_Votes DESC) AS posicao_unica,
    RANK() OVER (ORDER BY IMDB_Rating DESC) AS rank_com_gaps,
    DENSE_RANK() OVER (ORDER BY IMDB_Rating DESC) AS rank_sem_gaps
FROM read_csv_auto('{dataset_path}')
WHERE Released_Year ~ '^[0-9]{{4}}$'
ORDER BY nota DESC, votos DESC
LIMIT 20;
"""

resultado = conn.execute(query_global_ranking).fetchdf()
print("Top 20 Filmes - Rankings Comparativos:")
print(resultado.to_string(index=False))

Top 20 Filmes - Rankings Comparativos:
                                           titulo  ano  nota   votos  posicao_unica  rank_com_gaps  rank_sem_gaps
                         The Shawshank Redemption 1994   9.3 2343110              1              1              1
                                    The Godfather 1972   9.2 1620367              2              2              2
                                  The Dark Knight 2008   9.0 2303232              3              3              3
                           The Godfather: Part II 1974   9.0 1129952              4              3              3
                                     12 Angry Men 1957   9.0  689845              5              3              3
                                     Pulp Fiction 1994   8.9 1826188              6              6              4
    The Lord of the Rings: The Return of the King 2003   8.9 1642758              7              6              4
                                 Schindler's List

In [12]:
# 6.2 Ranking Particionado por Gênero
# Objetivo: Top 3 filmes de cada gênero principal
# Conceitos: PARTITION BY, filtragem de window function com subquery

query_top_by_genre = f"""
WITH ranked_movies AS (
    SELECT 
        SPLIT_PART(Genre, ',', 1) AS genero,
        Series_Title AS titulo,
        IMDB_Rating AS nota,
        Released_Year AS ano,
        RANK() OVER (
            PARTITION BY SPLIT_PART(Genre, ',', 1) 
            ORDER BY IMDB_Rating DESC
        ) AS rank_no_genero
    FROM read_csv_auto('{dataset_path}')
    WHERE Genre IS NOT NULL
)
SELECT 
    genero,
    titulo,
    nota,
    ano,
    rank_no_genero
FROM ranked_movies
WHERE rank_no_genero <= 3
ORDER BY genero, rank_no_genero;
"""

resultado = conn.execute(query_top_by_genre).fetchdf()

print("Top 3 Filmes por Gênero:")
print(resultado.to_string(index=False))

Top 3 Filmes por Gênero:
   genero                                            titulo  nota  ano  rank_no_genero
   Action                                   The Dark Knight   9.0 2008               1
   Action     The Lord of the Rings: The Return of the King   8.9 2003               2
   Action                                         Inception   8.8 2010               3
   Action The Lord of the Rings: The Fellowship of the Ring   8.8 2001               3
Adventure                                      Interstellar   8.6 2014               1
Adventure                                Back to the Future   8.5 1985               2
Adventure                              Inglourious Basterds   8.3 2009               3
Adventure                                          Das Boot   8.3 1981               3
Adventure                             2001: A Space Odyssey   8.3 1968               3
Adventure                                Lawrence of Arabia   8.3 1962               3
Adventure         

In [13]:
# 6.3 Ranking Particionado por Década
# Objetivo: Melhores filmes de cada década
# Conceitos: PARTITION BY com expressão calculada, DENSE_RANK
query_best_per_decade = f"""
WITH filmes_com_decada AS (
    SELECT 
        Series_Title AS titulo,
        CAST(Released_Year AS INTEGER) AS ano,
        FLOOR(CAST(Released_Year AS INTEGER) / 10) * 10 AS decada,
        IMDB_Rating AS nota,
        Director AS diretor,
        ROW_NUMBER() OVER (
            PARTITION BY FLOOR(CAST(Released_Year AS INTEGER) / 10) * 10
            ORDER BY IMDB_Rating DESC
        ) AS rank_decada
    FROM read_csv_auto('{dataset_path}')
    WHERE Released_Year ~ '^[0-9]{{4}}$'
)
SELECT 
    decada || 's' AS periodo,
    titulo,
    ano,
    nota,
    diretor,
    rank_decada
FROM filmes_com_decada
WHERE rank_decada <= 5
ORDER BY decada DESC, rank_decada;
"""

resultado = conn.execute(query_best_per_decade).fetchdf()
print("Top 5 Filmes por Década:")
print(resultado.to_string(index=False))

Top 5 Filmes por Década:
periodo                                            titulo  ano  nota              diretor  rank_decada
2020.0s                                          Hamilton 2020   8.6          Thomas Kail            1
2020.0s                                   Soorarai Pottru 2020   8.6        Sudha Kongara            2
2020.0s                                              Soul 2020   8.1          Pete Docter            3
2020.0s                                       Dil Bechara 2020   7.9       Mukesh Chhabra            4
2020.0s                        The Trial of the Chicago 7 2020   7.8         Aaron Sorkin            5
2010.0s                                         Inception 2010   8.8    Christopher Nolan            1
2010.0s                                      Gisaengchung 2019   8.6         Bong Joon Ho            2
2010.0s                                      Interstellar 2014   8.6    Christopher Nolan            3
2010.0s                                         

In [14]:
# 7. Análises com Viés de Negócio
# 7.1 Diretores Mais Consistentes
# Objetivo: Identificar diretores com maior consistência (menor desvio padrão nas notas)
# Conceitos: STDDEV, múltiplas agregações, HAVING para filtros complexos

query_consistent_directors = f"""
SELECT 
    Director AS diretor,
    COUNT(*) AS total_filmes,
    ROUND(AVG(IMDB_Rating), 2) AS nota_media,
    ROUND(MIN(IMDB_Rating), 2) AS nota_minima,
    ROUND(MAX(IMDB_Rating), 2) AS nota_maxima,
    ROUND(STDDEV(IMDB_Rating), 3) AS desvio_padrao
FROM read_csv_auto('{dataset_path}')
WHERE Director IS NOT NULL
GROUP BY Director
HAVING COUNT(*) >= 4
ORDER BY desvio_padrao ASC, nota_media DESC
LIMIT 15;
"""

resultado = conn.execute(query_consistent_directors).fetchdf()
print("Diretores Mais Consistentes (menor variação de qualidade):")
print(resultado.to_string(index=False))

Diretores Mais Consistentes (menor variação de qualidade):
         diretor  total_filmes  nota_media  nota_minima  nota_maxima  desvio_padrao
Andrei Tarkovsky             5        8.12          8.1          8.2          0.045
     Pete Docter             4        8.13          8.1          8.2          0.050
  Ingmar Bergman             7        8.14          8.1          8.2          0.053
  Matthew Vaughn             4        7.65          7.6          7.7          0.058
Federico Fellini             5        8.00          7.9          8.1          0.071
    Howard Hawks             7        7.86          7.8          8.0          0.079
  Alfonso Cuarón             6        7.75          7.6          7.9          0.122
   Neeraj Pandey             4        7.98          7.8          8.1          0.126
         Ang Lee             4        7.75          7.6          7.9          0.129
  Fred Zinnemann             4        7.75          7.6          7.9          0.129
     Woody Allen 

In [15]:
# 7.2 Evolução Temporal de Gêneros
# Objetivo: Analisar tendência de popularidade e qualidade de gêneros ao longo do tempo
# Conceitos: CTE, agregação temporal, ordenação complexa

query_genre_evolution = f"""
WITH genre_by_period AS (
    SELECT 
        SPLIT_PART(Genre, ',', 1) AS genero,
        CASE 
            WHEN CAST(Released_Year AS INTEGER) < 1980 THEN 'Clássico'
            WHEN CAST(Released_Year AS INTEGER) >= 1980 AND CAST(Released_Year AS INTEGER) < 2000 THEN 'Década de 80-90'
            ELSE 'Moderno (2000+)'
        END AS periodo,
        IMDB_Rating,
        No_of_Votes
    FROM read_csv_auto('{dataset_path}')
    WHERE Released_Year ~ '^[0-9]{{4}}$'
        AND Genre IS NOT NULL
)
SELECT 
    genero,
    periodo,
    COUNT(*) AS quantidade,
    ROUND(AVG(IMDB_Rating), 2) AS nota_media,
    ROUND(AVG(No_of_Votes), 0) AS votos_medios
FROM genre_by_period
GROUP BY genero, periodo
HAVING COUNT(*) >= 3
ORDER BY genero, 
    CASE periodo 
        WHEN 'Clássico' THEN 1 
        WHEN 'Década de 80-90' THEN 2 
        ELSE 3 
    END;
"""

resultado = conn.execute(query_genre_evolution).fetchdf()
print("Evolução de Gêneros ao Longo do Tempo:")
print(resultado.to_string(index=False))

Evolução de Gêneros ao Longo do Tempo:
   genero         periodo  quantidade  nota_media  votos_medios
   Action        Clássico          31        7.93      121451.0
   Action Década de 80-90          35        7.99      457764.0
   Action Moderno (2000+)         106        7.94      495242.0
Adventure        Clássico          27        7.99      170600.0
Adventure Década de 80-90          11        7.96      313807.0
Adventure Moderno (2000+)          33        7.90      431785.0
Animation        Clássico           5        7.68       68194.0
Animation Década de 80-90          26        7.97      246542.0
Animation Moderno (2000+)          51        7.93      298580.0
Biography        Clássico          15        7.95       94069.0
Biography Década de 80-90          22        7.97      271465.0
Biography Moderno (2000+)          51        7.92      325953.0
   Comedy        Clássico          50        7.98      105606.0
   Comedy Década de 80-90          46        7.82      180816.0
 

In [16]:
# 7.3 Análise de ROI Potencial (Receita vs Avaliação)
# Objetivo: Correlacionar sucesso comercial com crítica
# Conceitos: Filtragem de valores nulos, categorização com CASE

query_commercial_success = f"""
WITH base AS (
    SELECT
        Series_Title AS titulo,
        Released_Year AS ano,
        IMDB_Rating AS nota_imdb,
        Meta_score AS metascore,
        CAST(REPLACE(Gross, ',', '') AS BIGINT) AS receita_bruta
    FROM read_csv_auto('{dataset_path}')
    WHERE Gross IS NOT NULL
      AND IMDB_Rating IS NOT NULL
      AND Released_Year ~ '^[0-9]{{4}}$'
)
SELECT
    titulo,
    ano,
    nota_imdb,
    metascore,
    receita_bruta,
    CASE
        WHEN nota_imdb >= 8.0 AND receita_bruta > 100000000
            THEN 'Sucesso Crítico e Comercial'
        WHEN nota_imdb >= 8.0 AND receita_bruta <= 100000000
            THEN 'Sucesso Crítico'
        WHEN nota_imdb < 8.0 AND receita_bruta > 100000000
            THEN 'Sucesso Comercial'
        ELSE 'Performance Moderada'
    END AS categoria_sucesso
FROM base
ORDER BY receita_bruta DESC
LIMIT 25;
"""

resultado = conn.execute(query_commercial_success).fetchdf()

print("Análise de Sucesso Crítico vs Comercial:")
print(resultado.to_string(index=False))

Análise de Sucesso Crítico vs Comercial:
                                       titulo  ano  nota_imdb  metascore  receita_bruta           categoria_sucesso
   Star Wars: Episode VII - The Force Awakens 2015        7.9         80      936662225           Sucesso Comercial
                            Avengers: Endgame 2019        8.4         78      858373000 Sucesso Crítico e Comercial
                                       Avatar 2009        7.8         83      760507625           Sucesso Comercial
                       Avengers: Infinity War 2018        8.4         68      678815482 Sucesso Crítico e Comercial
                                      Titanic 1997        7.8         75      659325379           Sucesso Comercial
                                 The Avengers 2012        8.0         69      623279547 Sucesso Crítico e Comercial
                                Incredibles 2 2018        7.6         80      608581744           Sucesso Comercial
                              T

In [17]:
# 7.4 Identificação de Tendências Emergentes
# Objetivo: Gêneros com crescimento de qualidade em anos recentes
# Conceitos: CTEs aninhadas, comparação entre períodos, cálculo de variação percentual

query_emerging_trends = f"""
WITH base AS (
    SELECT
        SPLIT_PART(Genre, ',', 1) AS genero,
        CAST(Released_Year AS INTEGER) AS ano,
        IMDB_Rating AS nota_imdb
    FROM read_csv_auto('{dataset_path}')
    WHERE Released_Year ~ '^[0-9]{{4}}$'
      AND Genre IS NOT NULL
      AND IMDB_Rating IS NOT NULL
),
recent_stats AS (
    SELECT
        genero,
        AVG(nota_imdb) AS nota_media_recente,
        COUNT(*) AS qtd_recente
    FROM base
    WHERE ano >= 2010
    GROUP BY genero
),
older_stats AS (
    SELECT
        genero,
        AVG(nota_imdb) AS nota_media_antiga,
        COUNT(*) AS qtd_antiga
    FROM base
    WHERE ano < 2010
    GROUP BY genero
)
SELECT
    r.genero,
    ROUND(o.nota_media_antiga, 2) AS nota_pre_2010,
    ROUND(r.nota_media_recente, 2) AS nota_pos_2010,
    ROUND(r.nota_media_recente - o.nota_media_antiga, 2) AS variacao,
    ROUND(
        CASE 
            WHEN o.nota_media_antiga = 0 THEN NULL
            ELSE (r.nota_media_recente - o.nota_media_antiga)
                 / o.nota_media_antiga * 100
        END
    , 1) AS variacao_percentual,
    r.qtd_recente AS filmes_recentes,
    o.qtd_antiga AS filmes_antigos
FROM recent_stats r
INNER JOIN older_stats o
    ON r.genero = o.genero
WHERE r.qtd_recente >= 5
  AND o.qtd_antiga >= 5
ORDER BY variacao DESC;
"""

resultado = conn.execute(query_emerging_trends).fetchdf()

print("Gêneros com Maior Crescimento de Qualidade (Pré vs Pós 2010):")
print(resultado.to_string(index=False))

Gêneros com Maior Crescimento de Qualidade (Pré vs Pós 2010):
   genero  nota_pre_2010  nota_pos_2010  variacao  variacao_percentual  filmes_recentes  filmes_antigos
Biography           7.93           7.95      0.02                  0.2               29              59
   Comedy           7.90           7.90     -0.01                 -0.1               32             123
   Action           7.95           7.94     -0.01                 -0.1               55             117
Animation           7.93           7.92     -0.01                 -0.1               29              53
Adventure           7.94           7.93     -0.01                 -0.1               15              56
    Drama           7.97           7.92     -0.05                 -0.7               66             223
    Crime           8.03           7.97     -0.05                 -0.7               18              89
